# Preprocessing Notes

I believe there are 2 crucial components to this experiment that are **more important** than type of Neural Network:
- collection of good data with various scenarios
- preporcesing of the images to remove unnecessary training burden

## Steps 

- Crop the images
- Apply mask to leave only essential data
- Grayscale (see "Conclusions and assumptions")
- Blur the image to remove pixelation and smooth the embankments
- POSSIBLY: Detect "Canny Edges"
- POSSIBLY: Use Hugh algorithm to connect dots (draw lines)
- POSSIBLY: throw out **everything** else except left/right lanes
- Scale down the images to 28x28 if possible
- Visually verify all kinds of landscapes 
    - if I (human) can tell immediately how to steer then I can teach Neural Network to do so, too.

In [53]:
data_dir = "../../../DATA/behavioral_cloning_data/"
import csv

# TODO implement batch_from, batch_to - I did not need it for 8037 rows
# TODO implement has_header_row
def read_csv(file_path):
    # Opening spreadsheet to read in TEXT mode: 'rt'
    with open(file_path, 'rt') as csvfile:
        # Most common format of CSV, TODO improve
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"') 
        row_counter = 0
        headers = []
        data = []
        for row in spamreader:
            row_counter = row_counter + 1
            
            if row_counter == 1:
                headers = row
                # print (type(row))
                # print ( row)
                # print ('\t '.join(row))
            elif 1 < row_counter < 3:
                # print (type(row))
                # print ( row)
                # print ('\t '.join(row))
                data.append(row)
            else:
                data.append(row)
        print("row_counter", row_counter)
        return headers, data
    
# TEST:    
headers, data = read_csv(file_path = data_dir + 'driving_log.csv')
print("headers \n",headers)
print("3rd row of data \n",data[2:3])

row_counter 8037
headers 
 ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
3rd row of data 
 [['IMG/center_2016_12_01_13_31_12_937.jpg', ' IMG/left_2016_12_01_13_31_12_937.jpg', ' IMG/right_2016_12_01_13_31_12_937.jpg', ' 0', ' 0', ' 0', ' 1.453011']]


In [47]:
column_image_center = 0
column_image_left = 1
column_image_right = 2
column_steering = 3
column_throttle = 4
column_brake = 5
column_speed = 6
record_number = 1
print(data[record_number][column_speed])

def get_all_values(matrix, column_index):
    return [row[column_index] for row in matrix]

#TEST:
images_left = get_all_values(data, column_image_left)
print("print 3rd left image", images_left[2:3]) 

 21.87963
print 3rd left image [' IMG/left_2016_12_01_13_31_12_937.jpg']
